In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)


In [ ]:
import sys
sys.executable
sys.path

Here we need a quick work around as the gwas ready object does not have the cell_type annotation, we export it from the processed object with the highly variable genes

In [ ]:
adata = sc.read_h5ad("../data/current/annotated_expression/hca_heart_global_ctl200606_GWAS_RAW.h5ad")

In [ ]:
adata.shape

In [ ]:
adata.obs.head()

In [ ]:
adata.obs['cell_states'].cat.categories

We have an updated meta data table which should be used instead of the obs

In [ ]:
meta = pd.read_csv("../data/current/annotated_expression/hca_heart_global_ctl200601_metadata.csv", index_col=0)

In [ ]:
meta.head()

In [ ]:
np.shape(meta)

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
np.shape(adata)

## Use nuclei only to avoid batcheffects
Check which cell sources are currently present

In [ ]:
adata.obs.cell_source.value_counts()

In [ ]:
source = pd.Series(adata.obs.cell_source)
selected_cells = source.isin(['Sanger-Nuclei', 'Harvard-Nuclei'])
filtered = adata[selected_cells]

In [ ]:
np.shape(filtered)

Check out the cell annotations

In [ ]:
ct_count = filtered.obs.cell_type.value_counts()
ct_count

Also checkout the cluster annotations

In [ ]:
region_count = filtered.obs.region.value_counts()
region_count

In [ ]:
pd.crosstab(filtered.obs.region, filtered.obs.cell_states)

For magma we need to compute cell type averages (log counts) and the overall expression average. We will do this per anatomical region and for all regions at the same time on (cell type x region) annotations.

In [ ]:
filtered.var.head()

In [ ]:
## magma uses entrez ids so we need to check the id mapping (based on bioconductor annotations)
symbol2entrez = pd.read_table("../results/current/magma/symbol2entrez.txt", index_col="ENTREZID")
symbol2entrez.SYMBOL.nunique()

In [ ]:
# filtered.var.index.nunique()

In [ ]:
# pd.Series(filtered.var.index.isin(symbol2entrez.SYMBOL)).value_counts()

In [ ]:
# missing = list(set(filtered.var.index) - set(symbol2entrez.SYMBOL))
# missing

In [ ]:
## try with aliases instead
# alias2entrez = pd.read_table("../results/current/magma/alias2entrez.txt", index_col="ENTREZID")
# pd.Series(filtered.var.index.isin(alias2entrez.ALIAS)).value_counts()


In [ ]:
# missing = list(set(filtered.var.index) - set(alias2entrez.ALIAS))
# missing

It seems that a lot of non coding RNAs are selected and they do not have entrez gene ids, so we will ignore them for now

## Stick to gene symbols
There was a problem with the aliases: it leads to duplicated entrez ids, which break the downstream processing

## Create expression estimates per region and cell type

In [ ]:
celltypes = ct_count.axes[0].tolist()
cn1 = celltypes * len(region_count.axes[0].tolist())
cn2 = np.repeat(region_count.axes[0].tolist(), len(celltypes))
colnames = ["%s_%s" % (cn2[i], cn1[i]) for i in range(len(cn1))]
colnames.append("Average")
celltypes.append("Average")

avg = pd.DataFrame(index=filtered.var.index, columns=colnames)
# print(np.shape(avg))

for region in region_count.axes[0].tolist():
    region_expr = filtered[filtered.obs.region == region]
    # print(np.shape(region_expr))
    per_region = pd.DataFrame(index=region_expr.var.index, columns=celltypes)
    
    ## compute per cell type per region average expression
    for ct in ct_count.axes[0].tolist():
        cn = "%s_%s" % (region, ct)
        print(cn)
        # print(pd.Series(region_expr.obs.cell_type == ct).value_counts())
        tmp = region_expr[region_expr.obs.cell_type == ct]
        # print(np.shape(tmp.X))
        ct_mean = tmp.X.mean(axis=0)
        # print(np.shape(ct_mean))
        # ct_mean = region_expr.X[region_expr.obs.cell_type == ct].mean(axis=0)
        avg[cn] = np.transpose(ct_mean)
        per_region[ct] = np.transpose(ct_mean)
    
    ## comput per region average expression
    all_mean = region_expr.X.mean(axis=0)
    per_region["Average"] = np.transpose(all_mean)
    fname = "../results/current/magma/expr_per_region_%s.txt" % region
    
    ## replace gene symbols with entrez ids
    per_region = symbol2entrez.merge(per_region, left_on="SYMBOL", right_index=True)
    per_region = per_region.drop(["SYMBOL"], axis=1)
    per_region.to_csv(fname, sep="\t", index_label="GENE")

## compute overall average expression
avg["Average"] = np.transpose(filtered.X.mean(axis=0))
## replace gene symbols with entrez ids
avg = symbol2entrez.merge(avg, left_on="SYMBOL", right_index=True)
avg = avg.drop(["SYMBOL"], axis=1)
avg.to_csv("../results/current/magma/expr_all.txt", sep="\t", index_label="GENE")

In [ ]:
np.shape(avg)
np.shape(filtered)

## Create a similar matrix also across all regions

In [ ]:
celltypes = ct_count.axes[0].tolist()
avg_ct = pd.DataFrame(index=filtered.var.index, columns=celltypes)
    
for ct in ct_count.axes[0].tolist():
    print(ct)
    ct_mean = filtered[filtered.obs.cell_type == ct].X.mean(axis=0)
    avg_ct[ct] = np.transpose(ct_mean)

## compute overall average expression
avg_ct["Average"] = np.transpose(filtered.X.mean(axis=0))
## replace gene symbols with entrez ids
avg_ct = symbol2entrez.merge(avg_ct, left_on="SYMBOL", right_index=True)
avg_ct = avg_ct.drop(["SYMBOL"], axis=1)
avg_ct.to_csv("../results/current/magma/hvg_expr_crossregions_all.txt", sep="\t", index_label="GENE")

In [ ]:
celltypes = ct_count.axes[0].tolist()
avg_ct = pd.DataFrame(index=filtered.var.index, columns=celltypes)
    
for ct in ct_count.axes[0].tolist():
    print(ct)
    ct_mean = np.log(filtered[filtered.obs.cell_type == ct].raw.X.todense() + 1.0).mean(axis=0)
    avg_ct[ct] = np.transpose(ct_mean)

## compute overall average expression
avg_ct["Average"] = np.transpose(np.log(filtered.raw.X.todense() + 1.0).mean(axis=0))
## replace gene symbols with entrez ids
avg_ct = symbol2entrez.merge(avg_ct, left_on="SYMBOL", right_index=True)
avg_ct = avg_ct.drop(["SYMBOL"], axis=1)
avg_ct.to_csv("../results/current/magma/expr_crossregions_all.txt", sep="\t", index_label="GENE")

## Use cell states or clusters as annotations


In [ ]:
#cs_count = filtered.obs.cell_states.value_counts()
#pd.crosstab(filtered.obs.cell_states, filtered.obs.cell_type)

In [ ]:
pd.crosstab(filtered.obs.cell_states, filtered.obs.cell_states)

In [ ]:
cl_count = filtered.obs.cell_states.value_counts()
clusters = cl_count.axes[0].tolist()
avg_cl = pd.DataFrame(index=filtered.var.index, columns=clusters)
    
for cl in cl_count.axes[0].tolist():
    print(cl)
    cl_mean = filtered[filtered.obs.cell_states == cl].X.mean(axis=0)
    avg_cl[cl] = np.transpose(cl_mean)

## compute overall average expression
avg_cl["Average"] = np.transpose(filtered.X.mean(axis=0))
## replace gene symbols with entrez ids
avg_cl = symbol2entrez.merge(avg_cl, left_on="SYMBOL", right_index=True)
avg_cl = avg_cl.drop(["SYMBOL"], axis=1)
avg_cl.to_csv("../results/current/magma/expr_crossregions_cell_states.txt", sep="\t", index_label="GENE")

# Gene expression from the merged subclustering
Here we use a merged annotation table that was extracted from each of the subclustering objects (expression_for_magma_subclusters.ipynb) and subsequently merged (magma.Rmd).

First we try to merge this with the obs table in the adata object.

In [ ]:
merged_ann = pd.read_table("../data/current/annotated_expression/merged_metadata.txt", index_col="index")
merged_ann.head()

In [ ]:
adata.obs.head()

In [ ]:
np.shape(adata)

In [ ]:
merged = adata.obs.copy()
merged = merged.merge(merged_ann, how="left", left_index=True, right_index=True)
np.shape(merged)

In [ ]:
pd.Series(adata.obs.index.isin(merged_ann.index)).value_counts()

We are missing quite a few barcodes, so we need to check where this comes from!

In [ ]:
missing_barcodes = list(set(merged_ann.index) - set(adata.obs.index))
missing_barcodes[0:10]

In [ ]:
new_index_subcl = []
for i in merged_ann.index:
    new_index_subcl.append(("-").join(i.split("-")[:5]))
pd.Series(new_index_subcl).value_counts().value_counts()

In [ ]:
pd.Series(merged_ann.index.isin(adata.obs.index)).value_counts()

In [ ]:
missing_barcodes = list(set(adata.obs.index) - set(merged_ann.index))
missing_barcodes[0:10]

Try to match.. we first have a look at the suffixes used

In [ ]:
suffixes_global = []
suffixes_short_global = []
for i in missing_barcodes:
    suffixes_global.append(("-").join(i.split("-")[2:]))
    suffixes_short_global.append(("-").join(i.split("-")[2:5]))
pd.Series(suffixes_global).value_counts()

In [ ]:
pd.Series(pd.Series(suffixes_short_global).isin(pd.Series(suffixes_short_subcl))).value_counts()

It looks like this is doing the trick. Now we need to check if the shorter indices are still unique..

In [ ]:
short_indices_subcl = []
for i in merged_ann.index:
    short_indices_subcl.append(("-").join(i.split("-")[:5]))
    
short_indices_global = []
for i in adata.obs.index:
    short_indices_global.append(("-").join(i.split("-")[:5]))

pd.Series(pd.Series(short_indices_subcl).value_counts()).value_counts()

The majority is actually not unique, so we cannot simply add this as a column to the obs data frame. Instead we check by annotation later.. 

To do so, we add the new short indices to the adata.obs and the merge annotation data frames.

In [ ]:
merged_ann = merged_ann.assign(short_index=short_indices_subcl)
adata.obs = adata.obs.assign(short_index=short_indices_global)

In [ ]:
source = pd.Series(adata.obs.cell_source)
selected_cells = source.isin(['Sanger-Nuclei', 'Harvard-Nuclei'])
filtered = adata[selected_cells]
np.shape(filtered)

In [ ]:
## magma uses entrez ids so we need to check the id mapping (based on bioconductor annotations)
symbol2entrez = pd.read_table("../results/current/magma/symbol2entrez.txt", index_col="ENTREZID")
symbol2entrez.SYMBOL.nunique()

In [ ]:
filtered.var.index.nunique()

In [ ]:
pd.Series(filtered.var.index.isin(symbol2entrez.SYMBOL)).value_counts()

## Create gene expression matrix across all anatomical regions

In [ ]:
import functools
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

ann_name = pd.Series(paste(merged_ann.annotation, merged_ann.subtype, sep=":")).unique()
ann_name[0:10]

In [ ]:
subtypes = list(ann_name)
subtypes.append("Average")

avg_st = pd.DataFrame(index=filtered.var.index, columns=subtypes)
    
for st in ann_name:
    print(st)
    annotation = str(st).split(":")[0]
    subtype = str(st).split(":")[1]
    print("annotation: %s, subtype: %s" % (annotation, subtype))
    selected_indices = merged_ann[(merged_ann.annotation == annotation) & (merged_ann.subtype == subtype)].short_index
    selected_cells = filtered.obs.short_index.isin(selected_indices)
    print("Number of cells: %s" % np.sum(selected_cells))
    if (np.sum(selected_cells) == 0):
        continue
    st_mean = filtered[selected_cells].X.mean(axis=0)
    avg_st[st] = np.transpose(st_mean)

## compute overall average expression
avg_st["Average"] = np.transpose(filtered.X.mean(axis=0))
## replace gene symbols with entrez ids
avg_st = symbol2entrez.merge(avg_st, left_on="SYMBOL", right_index=True)
avg_st = avg_st.drop(["SYMBOL"], axis=1)
avg_st.to_csv("../results/current/magma/expr_crossregions_subtypes.txt", sep="\t", index_label="GENE")

In [ ]:
np.sum(selected_cells) == 0

In [ ]:
st = "stromal:EC23*"
annotation = str(st).split(":")[0]
subtype = str(st).split(":")[1]
print("annotation: %s, subtype: %s" % (annotation, subtype))
selected_indices = merged_ann[(merged_ann.annotation == annotation) & (merged_ann.subtype == subtype)].short_index
np.sum(filtered.obs.short_index.isin(selected_indices))